<a href="https://colab.research.google.com/github/ranedigamber/BigData/blob/main/Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os


In [ ]:
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.0'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION'] =spark_version

In [ ]:
# Install Spakr and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done


In [ ]:
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

In [ ]:
# Start a SparkSession
import findspark
findspark.init()

In [ ]:
#Start Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [ ]:
dataframe = spark.createDataFrame([
                                    (0, "Here is our DataFrame"),
                                    (1, "We are making one from scratch"),
                                    (2, "This will look very similar to a Panda DataFrame")
], ["id", "words"])
dataframe.show()

+---+--------------------+
| id|               words|
+---+--------------------+
|  0|Here is our DataF...|
|  1|We are making one...|
|  2|This will look ve...|
+---+--------------------+



In [ ]:
#Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-online/module_16/food.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("food.csv"), sep=",", header=True)   

In [ ]:
df.show()

+-------+-----+
|   food|price|
+-------+-----+
|  pizza|    0|
|  sushi|   12|
|chinese|   10|
+-------+-----+



In [ ]:
df.columns  

['food', 'price']

In [ ]:
df.describe()

DataFrame[summary: string, food: string, price: string]

In [ ]:
#Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [ ]:
# Next we need to create the list of struct fields
schema = [StructField("food", StringType(), True), StructField("price", IntegerType(), True),]
schema

[StructField('food', StringType(), True),
 StructField('price', IntegerType(), True)]

In [ ]:
#Pass in our fields
final = StructType(schema)
final

StructType([StructField('food', StringType(), True), StructField('price', IntegerType(), True)])

In [ ]:
#Read or data with our new schema
dataframe = spark.read.csv(SparkFiles.get("food.csv"), schema=final, sep=",", header=True)
dataframe.printSchema()

root
 |-- food: string (nullable = true)
 |-- price: integer (nullable = true)



In [ ]:
dataframe['price']
type(dataframe['price'])
dataframe.select('price')
type(dataframe.select('price'))
dataframe.select('price').show()

+-----+
|price|
+-----+
|    0|
|   12|
|   10|
+-----+



In [ ]:
#Add new colum
dataframe.withColumn('new price', dataframe['price']).show()

+-------+-----+---------+
|   food|price|new price|
+-------+-----+---------+
|  pizza|    0|        0|
|  sushi|   12|       12|
|chinese|   10|       10|
+-------+-----+---------+



In [ ]:
#Update column name
dataframe.withColumnRenamed('price', 'newer price').show()

+-------+-----------+
|   food|newer price|
+-------+-----------+
|  pizza|          0|
|  sushi|         12|
|chinese|         10|
+-------+-----------+



In [ ]:
# Double the price
dataframe.withColumn('doubleprice', dataframe['price']*2).show()

+-------+-----+-----------+
|   food|price|doubleprice|
+-------+-----+-----------+
|  pizza|    0|          0|
|  sushi|   12|         24|
|chinese|   10|         20|
+-------+-----+-----------+



In [ ]:
#Add a dollar to price
dataframe.withColumn('increased price', dataframe['price']+1).show()

+-------+-----+---------------+
|   food|price|increased price|
+-------+-----+---------------+
|  pizza|    0|              1|
|  sushi|   12|             13|
|chinese|   10|             11|
+-------+-----+---------------+



In [ ]:
#Half the price
dataframe.withColumn('halved price', dataframe['price']/2).show()

+-------+-----+------------+
|   food|price|halved price|
+-------+-----+------------+
|  pizza|    0|         0.0|
|  sushi|   12|         6.0|
|chinese|   10|         5.0|
+-------+-----+------------+



In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-online/module_16/wine.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("wine.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20

In [ ]:
#Order a Dataframe by ascending values
df.orderBy(df["points"].desc()).show()

+-------+--------------------+--------------------+------+-----+------------+--------------------+---------------+--------------------+--------------------+
|country|         description|         designation|points|price|    province|            region_1|       region_2|             variety|              winery|
+-------+--------------------+--------------------+------+-----+------------+--------------------+---------------+--------------------+--------------------+
|     US|The only one of t...|   Cailloux Vineyard|    99|   65|      Oregon|Walla Walla Valle...|   Oregon Other|               Syrah|              Cayuse|
| France|A magnificent Cha...|Dom P̩rignon Oeno...|    99|  385|   Champagne|           Champagne|           null|     Champagne Blend|     Mo��t & Chandon|
|     US|Depth and texture...|Royal City Stoner...|    99|  140|  Washington|Columbia Valley (WA)|Columbia Valley|               Syrah|       Charles Smith|
| France|98-100 Barrel sam...|       Barrel sample|    99|

In [ ]:
# Get top50 rows for values in ascending order on "points" column
df.orderBy(df["points"].asc()).show(50)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             country|         description|         designation|              points|               price|            province|            region_1|            region_2|             variety|              winery|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                  US|               Ripe |                null|                null|                null|                null|                null|                null|                null|                null|
|                  US|Brisk and clean, ...|                null|                null|                null|                null|                null|    

In [ ]:
# Import functions
from pyspark.sql.functions import avg
df.select(avg("points")).show() 

+-----------------+
|      avg(points)|
+-----------------+
|87.88834105383143|
+-----------------+



In [ ]:
from pyspark.sql.functions import sum
df.select(sum('points')).show()

+-----------+
|sum(points)|
+-----------+
|1.3262087E7|
+-----------+



In [ ]:
#Filter
df.filter("price<20").show(10)

+--------+--------------------+--------------------+------+-----+----------------+--------------------+--------------------+------------------+--------------------+
| country|         description|         designation|points|price|        province|            region_1|            region_2|           variety|              winery|
+--------+--------------------+--------------------+------+-----+----------------+--------------------+--------------------+------------------+--------------------+
|Bulgaria|This Bulgarian Ma...|             Bergul̩|    90|   15|        Bulgaria|                null|                null|            Mavrud|        Villa Melnik|
|   Spain|Earthy plum and c...|              Amandi|    90|   17|         Galicia|       Ribeira Sacra|                null|           Menc�_a|      Don Bernardino|
|      US|There's a lot to ...|                null|    90|   18|      California|Russian River Valley|              Sonoma|        Chardonnay|            De Loach|
|      US|

In [ ]:
#Filter by price on certain columns
df.filter("price<20").select(["points", "country", "variety", "winery"]).show()

+------+---------+--------------------+--------------------+
|points|  country|             variety|              winery|
+------+---------+--------------------+--------------------+
|    90| Bulgaria|              Mavrud|        Villa Melnik|
|    90|    Spain|             Menc�_a|      Don Bernardino|
|    90|       US|          Chardonnay|            De Loach|
|    91|       US|          Pinot Gris|   Trinity Vineyards|
|    91| Portugal|      Portuguese Red|Adega Cooperativa...|
|    86|       US|  Cabernet Sauvignon|      Belle Ambiance|
|    86| Portugal|                Baga| Adega de Cantanhede|
|    86|       US|          Chardonnay|            Parducci|
|    86| Portugal|      Portuguese Red|    Quinta do Portal|
|    86|   France|       Malbec-Merlot|               Rigal|
|    86|       US|          Chardonnay|     The Naked Grape|
|    86|   France|       Merlot-Malbec|   Georges Vigouroux|
|    86|   France|Ugni Blanc-Colombard|   Georges Vigouroux|
|    86|       US|      

In [71]:
df.select(["points", "country", "variety"]).show()

+------+-------+------------------+
|points|country|           variety|
+------+-------+------------------+
|    96|     US|Cabernet Sauvignon|
|    96|  Spain|     Tinta de Toro|
|    96|     US|   Sauvignon Blanc|
|    96|     US|        Pinot Noir|
|    95| France|Provence red blend|
|    95|  Spain|     Tinta de Toro|
|    95|  Spain|     Tinta de Toro|
|    95|  Spain|     Tinta de Toro|
|    95|     US|        Pinot Noir|
|    95|     US|        Pinot Noir|
|    95|  Italy|          Friulano|
|    95|     US|        Pinot Noir|
|    95|     US|        Pinot Noir|
|    95| France|            Tannat|
|    95|     US|        Pinot Noir|
|    95|     US|        Chardonnay|
|    95|     US|Cabernet Sauvignon|
|    95|  Spain|       Tempranillo|
|    95| France|            Malbec|
|    95|     US|        Pinot Noir|
+------+-------+------------------+
only showing top 20 rows



In [ ]:
df.filter(df["price"]<20).show()

+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------------------+--------------------+--------------------+
|  country|         description|         designation|points|price|        province|            region_1|            region_2|             variety|              winery|
+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------------------+--------------------+--------------------+
| Bulgaria|This Bulgarian Ma...|             Bergul̩|    90|   15|        Bulgaria|                null|                null|              Mavrud|        Villa Melnik|
|    Spain|Earthy plum and c...|              Amandi|    90|   17|         Galicia|       Ribeira Sacra|                null|             Menc�_a|      Don Bernardino|
|       US|There's a lot to ...|                null|    90|   18|      California|Russian River Valley|              Sonoma|          Chardonnay|            De

In [ ]:
df.filter(df["country"] == "US").show(10)

+-------+--------------------+--------------------+------+-----+----------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|  province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+----------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20 mon...|             Reserve|    96|   65|    Oregon|   Willamette Valley|Willamette Valley|        Pinot Noir|               Ponzi|
|     US|This re-named vin...|              Silice|    95|   65|    Or

In [ ]:
df.filter((df["price"]>15) & (df["province"]=="California")).show(5)

+-------+--------------------+--------------------+------+-----+----------+--------------------+--------+------------------+------------+
|country|         description|         designation|points|price|  province|            region_1|region_2|           variety|      winery|
+-------+--------------------+--------------------+------+-----+----------+--------------------+--------+------------------+------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|California|         Napa Valley|    Napa|Cabernet Sauvignon|       Heitz|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|California|      Knights Valley|  Sonoma|   Sauvignon Blanc|    Macauley|
|     US|The producer sour...|Gap's Crown Vineyard|    95|   60|California|        Sonoma Coast|  Sonoma|        Pinot Noir|   Blue Farm|
|     US|This blockbuster,...|     Rainin Vineyard|    95|  325|California|Diamond Mountain ...|    Napa|Cabernet Sauvignon|        Hall|
|     US|This fresh and li...|Gap'